# URLs para obtenção dos datasets (Obtido dia 07/06/2021)
- ### Explicação dos datasets: https://repositoriodatasharingfapesp.uspdigital.usp.br/
- ### Dataset do hospital Albert Einstein: https://repositoriodatasharingfapesp.uspdigital.usp.br/handle/item/98 - (2020-06-30)
- ### Dataset do hospital Sirio Libanes: https://repositoriodatasharingfapesp.uspdigital.usp.br/handle/item/97 (2020-06-30)
- ### Dataset do hospital Beneficencia Portuguesa: https://repositoriodatasharingfapesp.uspdigital.usp.br/handle/item/101 (2021-04-28)
- ### Dataset do grupo Fleury: https://repositoriodatasharingfapesp.uspdigital.usp.br/handle/item/99 - (2020-06-30)
- ### Dataset do hospital das clinicas da faculdade de medicina da Universidade de São Paulo: https://repositoriodatasharingfapesp.uspdigital.usp.br/handle/item/100 - (2021-02-17)

# 1. Importing Libraries

In [8]:
import json
from operator import itemgetter
import os
import sys

current_path = os.path.abspath(os.getcwd())
sys.path.append(f"{current_path}/../libs")

from plot import plot_confusion_matrix, plot_table
from config import load_config

# 2. Defining Constants and Globals

In [9]:
config = load_config("../config.yaml")

# 3. Auxiliar Functions

In [10]:
def get_avg_metrics(ml_results):
    accuracy = 0
    precision = 0
    recall = 0
    f1_score = 0
    tp = 0
    tn = 0
    n_hospitals = len(ml_results)
    for hospital, metrics in ml_results.items():
        accuracy += float(metrics["accuracy"])
        precision += float(metrics["precision"])
        recall += float(metrics["recall"])
        f1_score += float(metrics["f1-score"])
        tp += float(metrics["confusion matrix"]["tp"])
        tn += float(metrics["confusion matrix"]["tn"])
    return (round(accuracy/n_hospitals,3),
            round(precision/n_hospitals,3),
            round(recall/n_hospitals,3),
            round(f1_score/n_hospitals,3),
            tp,
            tn)

# 4. Reading Complete Dataset Feature Selection Report

In [11]:
with open(config.REPORT_PATH+'final-results.json', 'r') as fp:
    feature_selection_report = json.load(fp)

In [12]:
def get_std_var_metrics(ml_results,avg_acc,avg_prec,avg_rec,avg_f1):
    acc=0
    prec=0
    rec = 0
    f1 = 0
    for hospital, metrics in ml_results.items():
        acc += pow(abs(float(metrics["accuracy"])-avg_acc),2)
        prec += pow(abs(float(metrics["precision"])-avg_prec),2)
        rec += pow(abs(float(metrics["recall"])-avg_rec),2)
        f1 += pow(abs(float(metrics["f1-score"])-avg_f1),2)
    return (round(pow((acc/(len(ml_results))),1/2),3),
            round(pow((prec/(len(ml_results))),1/2),3),
            round(pow((rec/(len(ml_results))),1/2),3),
            round(pow((f1/(len(ml_results))),1/2),3))

In [13]:
def get_row(ml_results):
    n_hospitals = len(ml_results)
    avg_acc,avg_prec,avg_rec,avg_f1,tp,tn=get_avg_metrics(ml_results)
    std_var_acc,std_var_prec,std_var_rec,std_var_f1=get_std_var_metrics(ml_results,avg_acc,avg_prec,avg_rec,avg_f1)
    tp = round(tp/n_hospitals,3)
    tn = round(tn/n_hospitals,3)
    fp = round(1 - tp/n_hospitals,3)
    fn = round(1 - tn/n_hospitals,3)
    return [
        str(f"{avg_acc} +- {std_var_acc}"),
        str(f"{avg_prec} +- {std_var_prec}"),
        str(f"{avg_rec} +- {std_var_rec}"),
        str(f"{avg_f1} +- {std_var_f1}"),
        tp,
        tn,
        fp,
        fn,
    ]

In [14]:
for fs_model_name,fs_model in feature_selection_report.items():
    data = []
    columns = [
        "accuracy","precision","recall","f1-score",
        "tp","tn","fp","fn"]
    for ml_model_name, ml_model in fs_model.items(): 
        if ml_model_name in config.MODELS:
            data.append(get_row(ml_model))
    path = f"{config.PLOTS_PATH}final-result/"
    if not os.path.exists(path):
        os.makedirs(path)
    print(config.MODELS)
    plot_table(
        data,
        columns=columns, 
        rows=config.MODELS,
        path=f"{path}{fs_model_name}",
        title = (f"{fs_model_name} Results"))

['Decision Tree', 'Random Forest']
['Decision Tree', 'Random Forest']
['Decision Tree', 'Random Forest']
